# Linear Programming optimization tutorial datacamp

## Objectives
- Learning how to use PuLP
- Learn how to model LP problems in python

## The problem
- Cake bakery that produces and sells two types of cakes: A and B
- Cake bakery works on a 30 days / month schedulle
- Cake production is under the following constraints
  - Ovens: only one oven working 30 days/month
  - Bakers: 2 bakers working 30 days/month
  - Packers: 1 packer working 22 days/month

- Objectve: maximize profits of the bakery

In [2]:
import pulp as pl

In [21]:
# Xa: # of cakes of type A produced per month
# Xb: # of cakes of type B produced per month
Xa = pl.LpVariable("num_A_cakes", lowBound=0, cat='Integer')
Xb = pl.LpVariable("num_B_cakes", lowBound=0, cat='Integer')

In [22]:
# Define the optimization problem with the type of equation
problem = pl.LpProblem('bakery_monthly_profits', pl.LpMaximize)

In [23]:
# A simple expression (without equality) will be interpreted as the objective function
# Cakes A are sold at a $20 profit per unit, while cakes B are sold at $40 profit per unit
problem += 20*Xa + 40*Xb

In [24]:
# Inequality and equality expressions will be interpreted as constraints for the problem

# Ovens constraint 
problem += 0.5*Xa + 1.0*Xb <= 30

In [25]:
# Bakers constraint
problem += 1.0*Xa + 2.5*Xb <= 60

In [26]:
# Packers constraint
problem += 1.0*Xa + 2.0*Xb <= 22

In [27]:
# Feasibility space constraints
problem += Xa >= 0
problem += Xb >= 0

In [28]:
problem.constraints

OrderedDict([('_C1', 0.5*num_A_cakes + 1.0*num_B_cakes + -30.0 <= 0),
             ('_C2', 1.0*num_A_cakes + 2.5*num_B_cakes + -60.0 <= 0),
             ('_C3', 1.0*num_A_cakes + 2.0*num_B_cakes + -22.0 <= 0),
             ('_C4', 1*num_A_cakes + 0 >= 0),
             ('_C5', 1*num_B_cakes + 0 >= 0)])

In [30]:
problem.objective

20*num_A_cakes + 40*num_B_cakes + 0

In [31]:
# solves the problem
status = problem.solve()

In [33]:
# You can specify which solver you will use, but solver binaries should be installed in your path env
# status = problem.solve(solver=pl.GLPK())

In [35]:
pl.LpStatus[status]

'Optimal'

In [37]:
print(f"Optimal solution - Xa: {pl.value(Xa)}, Xb: {pl.value(Xb)}")

Optimal solution - Xa: 0.0, Xb: 11.0


# Glass manufacturer example

In [38]:
import pulp as lp
# Initialize Class
model = lp.LpProblem("Maximize Glass Co. Profits", lp.LpMaximize)

# Define variables
w = lp.LpVariable("wine_batches",lowBound=0, upBound=6, cat='Integer')
b = lp.LpVariable("beer_batches", lowBound=0, cat='Integer')

# Define the objective function
# Wine batch profit $5
# Beet batch profit $4.5
model += 5.0*w + 4.5*b

# Production capacity constraint 
# Total production <= 60 hours
# Wine glass batch takes 6 hours
# Beer glass batch takes 5 hours
model += 6*w + 5*b <= 60

# Warehouse capacity constraint
# Warehouse capacity <= 150 racks
# Wine glass batch takes 10 racks
# Beer glass batch takes 20 racks
model += 10*w + 20*b <= 150

# Production equipment constraints
# Wine glass and Beer glass variables are Integer values >= 0
model += w >= 0
model += b >= 0

# Order constraints
# Wine glass batches <= 6 batches
model += w <= 6

model.solve()

print("Optimization status: ", lp.LpStatus[model.status])
print("w: ", lp.value(w), "\tb: ", lp.value(b))

Optimization status:  Optimal
w:  6.0 	b:  4.0


C:\ProgramData\miniconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Glass manufacturer with LpSum
- What if there were a huge # of variables?

In [39]:
cake_types = ['A','B','C','D','E','F']
profit_by_cake = {'A':20,'B':40,'C':33,'D':14,'E':6,'F':60}

# generates variables based on cake_types variable
var_dict = dict([ (cake_type, lp.LpVariable(cake_type, lowBound=0, upBound=None, cat="Integer")) for cake_type in cake_types])

model =  lp.LpProblem("cake_opt_with_lpsum",lp.LpMaximize)

# Generates the objective function based on cake_types and a dictionary of profits
model += lp.lpSum([profit_by_cake[cake_type] * var_dict[cake_type] for cake_type in cake_types])

## Minimize transportation costs

In [40]:
import pulp as lp

# Initialize Model
model = lp.LpProblem("minimize_transportation_cots", lp.LpMinimize)

# Build the lists and the demand dictionary
warehouse = ['New York', 'Atlanta']
customers = ['East', 'South', 'Midwest', 'West']
regional_demand = [1800, 1200, 1100, 1000]

demand = dict(zip(customers, regional_demand))

In [41]:
costs = {
    ('New York', 'East'): 211,
    ('New York', 'South'): 232,
    ('New York', 'Midwest'): 240,
    ('New York', 'West'): 300,
    ('Atlanta', 'East'): 232,
    ('Atlanta', 'South'): 212,
    ('Atlanta', 'Midwest'): 230,
    ('Atlanta', 'West'): 280
}

In [46]:
var_dict  = dict(
    [((w,c), lp.LpVariable(w[:1].lower() + c[:1].lower(),lowBound=0, upBound=None, cat='Integer'))
     for c in customers
     for w in warehouse])
var_dict

{('New York', 'East'): ne,
 ('Atlanta', 'East'): ae,
 ('New York', 'South'): ns,
 ('Atlanta', 'South'): as,
 ('New York', 'Midwest'): nm,
 ('Atlanta', 'Midwest'): am,
 ('New York', 'West'): nw,
 ('Atlanta', 'West'): aw}

In [47]:
model += lp.lpSum([var_dict[(w,c)] * costs[(w,c)] 
                   for c in customers 
                   for w in warehouse])

In [48]:
model.objective

232*ae + 230*am + 212*as + 280*aw + 211*ne + 240*nm + 232*ns + 300*nw + 0

In [49]:
# Adding constraints
for c in customers:
    warehouse_offer_per_customer = lp.lpSum([var_dict[(w,c)] for w in warehouse])
    model += warehouse_offer_per_customer == demand[c]

In [50]:
model.constraints

OrderedDict([('_C1', 1*ae + 1*ne + -1800 = 0),
             ('_C2', 1*as + 1*ns + -1200 = 0),
             ('_C3', 1*am + 1*nm + -1100 = 0),
             ('_C4', 1*aw + 1*nw + -1000 = 0)])

In [51]:
model.solve()

1

In [52]:
lp.LpStatus[model.status]

'Optimal'

In [63]:
# Optimal offer per warehouse for each customer
[(wc, lp.value(var_dict[wc])) for wc in var_dict]

[(('New York', 'East'), 1800.0),
 (('Atlanta', 'East'), 0.0),
 (('New York', 'South'), 0.0),
 (('Atlanta', 'South'), 1200.0),
 (('New York', 'Midwest'), 0.0),
 (('Atlanta', 'Midwest'), 1100.0),
 (('New York', 'West'), 0.0),
 (('Atlanta', 'West'), 1000.0)]

# Case studies from PuLP documentation

## A Blending Problem
- Whiskas cat food is made from a blend of many cat food products as cheaply as possible

- Objective: Minimize costs
- Nutritional analysis constraints
- Chicken, beef, mutton, rice, wheat and Gel

- Each ingredient has protein, fat, fibre and salt in the final product

In [7]:
import pandas as pd
import pulp as lp

In [8]:
df_ingredients = pd.DataFrame(data = { "Protein":[0.1, 0.2, 0.15, 0, 0.04, 0],
                                       "Fat": [0.08, 0.1, 0.11, 0.010, 0.010, 0],
                                       "Fibre": [0.001, 0.005, 0.003, 0.1, 0.15, 0],
                                       "Salt": [0.002, 0.005, 0.007, 0.002, 0.008, 0],
                                       "Cost per g": [0.013, 0.008, 0.010, 0.002, 0.005, 0.001]},
                              index = ["Chicken","Beef","Mutton","Rice","Wheat","Gel"])

In [9]:
df_ingredients

,Protein,Fat,Fibre,Salt,Cost per g
Chicken,0.10,0.08,0.001,0.002,0.013
Beef,0.20,0.10,0.005,0.005,0.008
Mutton,0.15,0.11,0.003,0.007,0.010
Rice,0.00,0.01,0.100,0.002,0.002
Wheat,0.04,0.01,0.150,0.008,0.005
Gel,0.00,0.00,0.000,0.000,0.001


In [14]:
# Variables - ammount of grams of each ingredient in 100g blend
vars_dict = lp.LpVariable.dicts(name="ammount",
                                indices=["Chicken","Beef","Mutton","Rice","Wheat","Gel"],
                                lowBound=0,
                                upBound=100,
                                cat='Continuous')

In [15]:
costs = df_ingredients["Cost per g"].to_dict()

In [16]:
# Objective - minimize total cost of producing 100g blend (sum of variables * cost)
model = lp.LpProblem(name='minimize_blend_costs',sense=lp.LpMinimize)

# Add objective function
model += lp.lpSum([costs[ingredient] * vars_dict[ingredient] for ingredient in vars_dict])

In [17]:
protein_per_gram = df_ingredients["Protein"].to_dict()
fat_per_gram = df_ingredients["Fat"].to_dict()
fibre_per_gram = df_ingredients["Fibre"].to_dict()
salt_per_gram = df_ingredients["Salt"].to_dict()

In [18]:
# Constraints

# Blend should have 100g in total
model += lp.lpSum([vars_dict[ingredient] for ingredient in vars_dict]) == 100.0

# Blend should have at least 8g of protein
model += lp.lpSum([vars_dict[ingredient] * protein_per_gram[ingredient] for ingredient in vars_dict]) >= 8.0

# Blend should have at least 6g of fat
model += lp.lpSum([vars_dict[ingredient] * fat_per_gram[ingredient] for ingredient in vars_dict]) >= 6.0

# Blend should have at most 2g of fiber
model += lp.lpSum([vars_dict[ingredient] * fibre_per_gram[ingredient] for ingredient in vars_dict]) <= 2.0

# Blend should have at most 0.4g of salt
model += lp.lpSum([vars_dict[ingredient] * salt_per_gram[ingredient] for ingredient in vars_dict]) <= 0.4

In [19]:
model.objective

0.008*ammount_Beef + 0.013*ammount_Chicken + 0.001*ammount_Gel + 0.01*ammount_Mutton + 0.002*ammount_Rice + 0.005*ammount_Wheat + 0.0

In [20]:
model.constraints

OrderedDict([('_C1',
              1*ammount_Beef + 1*ammount_Chicken + 1*ammount_Gel + 1*ammount_Mutton + 1*ammount_Rice + 1*ammount_Wheat + -100.0 = 0),
             ('_C2',
              0.2*ammount_Beef + 0.1*ammount_Chicken + 0.15*ammount_Mutton + 0.04*ammount_Wheat + -8.0 >= 0),
             ('_C3',
              0.1*ammount_Beef + 0.08*ammount_Chicken + 0.11*ammount_Mutton + 0.01*ammount_Rice + 0.01*ammount_Wheat + -6.0 >= 0),
             ('_C4',
              0.005*ammount_Beef + 0.001*ammount_Chicken + 0.003*ammount_Mutton + 0.1*ammount_Rice + 0.15*ammount_Wheat + -2.0 <= 0),
             ('_C5',
              0.005*ammount_Beef + 0.002*ammount_Chicken + 0.007*ammount_Mutton + 0.002*ammount_Rice + 0.008*ammount_Wheat + -0.4 <= 0)])

In [21]:
model.solve()

1

In [22]:
lp.LpStatus[model.status]

'Optimal'

In [23]:
optimal_solution = dict([(variable.name, variable.value()) for variable in model.variables()])
optimal_solution

{'ammount_Beef': 60.0,
 'ammount_Chicken': 0.0,
 'ammount_Gel': 40.0,
 'ammount_Mutton': 0.0,
 'ammount_Rice': 0.0,
 'ammount_Wheat': 0.0}

In [26]:
print("Total cost of blend (optimal): ", model.objective.value())

Total cost of blend (optimal):  0.52


## Set Partitioning Problem
- Determine how items in one set S can be partitioned into smaller subsets
- All items in S must be contained in one and only one partition (MECE)
- Examples
  - Set packing - all items must be contained in zer or one partitions
  - Set covering - all items must be contained in at least one partition
  
- Wedding planner to determine guest seating allocations for a wedding
- Each table is a partition and the invited guests are the elements of set S
- Objective: maximize total happiness of the tables

### Solution 1
- Try out every single combination
- By law of large numbers this becomes quieckly unsolvable (without using column generation)
- Allows for non-linear relation (happiness) be modelled as a linear problem

In [27]:
max_tables = 5
max_table_size = 4
guests = "A B C D E F G I J K L M N O P Q R".split()

In [30]:
# create a list of all possible tables
possible_tables = [tuple(c) for c in lp.allcombinations(guests, max_table_size)]
len(possible_tables)

3213

In [31]:
# create a binary variable to state that a table setting is used
x = lp.LpVariable.dicts("table", possible_tables, lowBound=0, upBound=1, cat=lp.LpInteger)

In [32]:
def happiness(table):
    """
    Find the happiness of the table by calculating the maximum distance between letters
    """
    return abs(ord(table[0]) - ord(table[-1]))

In [33]:
# Defining the problem
model = lp.LpProblem("wedding_searing_model", lp.LpMinimize)

# Objective function
model += lp.lpSum([happiness(table) * x[table] for table in possible_tables])

In [35]:
# specifu the maximum number of tables
model += (
    lp.lpSum([x[table] for table in possible_tables]) <= max_tables,
    "Maximum_number_of_tables"
)

In [36]:
# A guest must be seated at one and only one table
for guest in guests:
    model += (
        lp.lpSum([x[table] for table in possible_tables if guest in table]) == 1,
        f"Must_seat_{guest}"
    )

In [37]:
model.solve()

1

In [40]:
optimal_solution = dict([(variable.name, variable.value() == 1.0) for variable in model.variables() if variable.value() == 1.0])
optimal_solution

{"table_('A',_'B',_'C',_'D')": True,
 "table_('E',_'F',_'G')": True,
 "table_('I',_'J',_'K',_'L')": True,
 "table_('M',_'N')": True,
 "table_('O',_'P',_'Q',_'R')": True}

## Sudoku Problem
- Sudoku is a problem with an incomplete 9x9 table of numbers that must be filled according to several rules
- Within any 3x3 boxes, each of numbers 1 to 9 must be found
- Within any column of 9x9 grid, each columns must have 1 to 9
- Within any row of the 9x9 grid, each of the numbers 1 to 9 must be found

This can be modeled as a LP problem

### Decision variables
- Create a integer variable from 1 to 9 for each of the 81 (9x9) squares
- Awfully complicated, there is no "not equal to" constraint in an LP problem (only lower than / higher than within bounds)
- We can ensure the sum of all the values in a box/row/column equal 45, but there are many solutions satisfying that (repeating numbers)
- Big M constraints: compare each pair of values in a box/column/row which number is strictly smaller than the other

- **chosen**: 
  - we create 729 binary variables, which represent 9 problem variables for each of 81 squares. 
  - In each square there are 9 variables that can assume true or false, but only one of the 9 can be true at each square
  
### Objective Function
- Sudoku: there is no solution better than another solution
- We are not trying to minimize or maximize anything
- We are just trying to find values on our variables
- We can use both `LpMinimize` or `LpMaximize`

### Constraints

- Sudoku problem known contraints
- One number per square (9 binary variables per square)
- Starting numbers of sudoku must be fixed / placed

In [41]:
import pulp as lp

In [42]:
# ROWS, COLS and VALUES form 1 to 9
VALS = ROWS = COLS = range(1,10)

In [43]:
# Boxes
Boxes = [
    [(3*box_num_row + row + 1, 3*box_num_col + col + 1) for row in range(3) for col in range(3)] 
     for box_num_row in range(3) 
     for box_num_col in range(3)
]
Boxes

[[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)],
 [(1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6)],
 [(1, 7), (1, 8), (1, 9), (2, 7), (2, 8), (2, 9), (3, 7), (3, 8), (3, 9)],
 [(4, 1), (4, 2), (4, 3), (5, 1), (5, 2), (5, 3), (6, 1), (6, 2), (6, 3)],
 [(4, 4), (4, 5), (4, 6), (5, 4), (5, 5), (5, 6), (6, 4), (6, 5), (6, 6)],
 [(4, 7), (4, 8), (4, 9), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9)],
 [(7, 1), (7, 2), (7, 3), (8, 1), (8, 2), (8, 3), (9, 1), (9, 2), (9, 3)],
 [(7, 4), (7, 5), (7, 6), (8, 4), (8, 5), (8, 6), (9, 4), (9, 5), (9, 6)],
 [(7, 7), (7, 8), (7, 9), (8, 7), (8, 8), (8, 9), (9, 7), (9, 8), (9, 9)]]

In [44]:
# problem
prob = lp.LpProblem("Sudoku")

In [45]:
# decition variables
choices = lp.LpVariable.dicts("Choice", (VALS, ROWS, COLS), cat=lp.LpBinary)

In [46]:
# we do not define an objective function since none is needed

In [48]:
# A constraint ensureing that only one value can be in each square is created
for row in ROWS:
    for col in COLS:
        prob += lp.lpSum([choices[val][row][col] for val in VALS]) == 1

In [50]:
# Only one occurrence of val can occur in each row, column and box

for val in VALS:
    for row in ROWS:
        prob += lp.lpSum([choices[val][row][col] for col in COLS] == 1)
    
    for col in COLS:
        prob += lp.lpSum([choices[val][row][col] for row in ROWS] == 1)
        
    for box in Boxes:
        prob += lp.lpSum([choices[val][row][col] for (row, col) in box]) == 1

In [51]:
# The starting numbers are entered as constraints
input_data = [
    (5, 1, 1),
    (6, 2, 1),
    (8, 4, 1),
    (4, 5, 1),
    (7, 6, 1),
    (3, 1, 2),
    (9, 3, 2),
    (6, 7, 2),
    (8, 3, 3),
    (1, 2, 4),
    (8, 5, 4),
    (4, 8, 4),
    (7, 1, 5),
    (9, 2, 5),
    (6, 4, 5),
    (2, 6, 5),
    (1, 8, 5),
    (8, 9, 5),
    (5, 2, 6),
    (3, 5, 6),
    (9, 8, 6),
    (2, 7, 7),
    (6, 3, 8),
    (8, 7, 8),
    (7, 9, 8),
    (3, 4, 9),
    (1, 5, 9),
    (6, 6, 9),
    (5, 8, 9),
]

In [52]:
for val, row, col in input_data:
    prob += choices[val][row][col] == 1

In [53]:
prob.writeLP("Sudoku.lp")

prob.solve()

print("Status; ", lp.LpStatus[prob.status])

Status;  Optimal


In [55]:
# A file called sudokuout.txt is created/overwritten for writing to
sudokuout = open("sudokuout.txt", "w")

# The solution is written to the sudokuout.txt file
for r in ROWS:
    if r in [1, 4, 7]:
        sudokuout.write("+-------+-------+-------+\n")
    for c in COLS:
        for v in VALS:
            if choices[v][r][c].value() == 1:
                if c in [1, 4, 7]:
                    sudokuout.write("| ")
                sudokuout.write(str(v) + " ")
                if c == 9:
                    sudokuout.write("|\n")
sudokuout.write("+-------+-------+-------+")
sudokuout.close()

### FInding all solutions
- by adding constraint that the last solution cannot be found

In [60]:
# A file called sudokuout.txt is created/overwritten for writing to
sudokuout = open("sudokuout.txt", "w")

while True:
    prob.solve()
    # The status of the solution is printed to the screen
    print("Status:", lp.LpStatus[prob.status])
    # The solution is printed if it was deemed "optimal" i.e met the constraints
    if lp.LpStatus[prob.status] == "Optimal":
        # The solution is written to the sudokuout.txt file
        for r in ROWS:
            if r in [1, 4, 7]:
                sudokuout.write("+-------+-------+-------+\n")
            for c in COLS:
                for v in VALS:
                    if lp.value(choices[v][r][c]) == 1:
                        if c in [1, 4, 7]:
                            sudokuout.write("| ")
                        sudokuout.write(str(v) + " ")
                        if c == 9:
                            sudokuout.write("|\n")
        sudokuout.write("+-------+-------+-------+\n\n")
        # The constraint is added that the same solution cannot be returned again
        prob += (
            lp.lpSum(
                [
                    choices[v][r][c]
                    for v in VALS
                    for r in ROWS
                    for c in COLS
                    if lp.value(choices[v][r][c]) == 1
                ]
            )
            <= 80
        )
    # If a new optimal solution cannot be found, we end the program
    else:
        break
sudokuout.close()

Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
Status: 

KeyboardInterrupt: 

## Transportation Problem
- Boutique with 2 warehouses to distribute beer to 5 bars
- Bars demand crates of beer weekly, which are then dispatched from warehouse
- Weekly optimizer for bar supply by warehouses
- Objective to minimize the cost of the whole operation
- Problem is also constrained to warehouse supply capacity

In [64]:
warehouses = ["A", "B"]
bars = [1,2,3,4,5]

warehouse_capacity = {"A": 1000, "B": 4000}

demand_per_bar = {
    1: 500,
    2: 900,
    3: 1800,
    4: 200,
    5: 700
}

In [66]:
# Problem definition 
model = lp.LpProblem("supply_cost_minimization",lp.LpMinimize)

# Variables - # of crates shipped from Warehouse X to Bar Y, with X in (A,B) and Y in (1,2,3,4,5)
x = lp.LpVariable.dicts("craters",[(warehouse, bar) for warehouse in warehouses for bar in bars], lowBound=0,cat='Integer')
x

{('A', 1): craters_('A',_1),
 ('A', 2): craters_('A',_2),
 ('A', 3): craters_('A',_3),
 ('A', 4): craters_('A',_4),
 ('A', 5): craters_('A',_5),
 ('B', 1): craters_('B',_1),
 ('B', 2): craters_('B',_2),
 ('B', 3): craters_('B',_3),
 ('B', 4): craters_('B',_4),
 ('B', 5): craters_('B',_5)}

### Costs and Objective function
- Usually the cost of operating a route has 2 components: one fixed, one variable
  - Fixed cost of one truck going from one place to another
  - Variable costs dependent on the # of items transported
- In these situations we can model fixed costs by using zero-one integer variables
- Let's simplify and assume there are only variable costs (US$/crate)

In [71]:
transp_costs = dict(zip([(warehouse, bar) for warehouse in warehouses for bar in bars],
                        [2,4,5,2,1,3,1,3,2,3]))
transp_costs

{('A', 1): 2,
 ('A', 2): 4,
 ('A', 3): 5,
 ('A', 4): 2,
 ('A', 5): 1,
 ('B', 1): 3,
 ('B', 2): 1,
 ('B', 3): 3,
 ('B', 4): 2,
 ('B', 5): 3}

In [72]:
# Objective function to be minimized - total cost of transport
model += lp.lpSum([transp_costs[(wh,bar)] * x[(wh,bar)] for wh in warehouses for bar in bars])

In [73]:
# Supply Constraints
for wh in warehouses:
    model += lp.lpSum([x[(wh,bar)] for bar in bars]) <= warehouse_capacity[wh]

# Demand Constraints
for bar in bars:
    model += lp.lpSum([x[(wh,bar)] for wh in warehouses]) == demand_per_bar[bar]

In [76]:
status = model.solve()
print("Model solution status: ", lp.LpStatus[status])

Model solution status:  Optimal


In [83]:
optimal_solution = dict([(x[var].name, x[var].value()) for var in x])
optimal_solution

{"craters_('A',_1)": 300.0,
 "craters_('A',_2)": 0.0,
 "craters_('A',_3)": 0.0,
 "craters_('A',_4)": 0.0,
 "craters_('A',_5)": 700.0,
 "craters_('B',_1)": 200.0,
 "craters_('B',_2)": 900.0,
 "craters_('B',_3)": 1800.0,
 "craters_('B',_4)": 200.0,
 "craters_('B',_5)": 0.0}

# Inbalance in supply and demand: storage
- Storage has costs and oftentimes we need to account for that
- We can do that by creating a "dummy" bar demand with costs associated with storage
- It is a virtual demand node that is only created to account for storage costs for each warehouse, which may be different.
- With excess demand the problem is infeasible, threfore this buffer node shoudl be exactly de difference `demand - supply` 

- To prevent for infeasible solutions when demand > supply, we can add a virtual node that represent buying from other distributors

In [102]:
warehouses = ["A", "B"]
bars = [1,2,3,4,5, 99] #99 is our dummy node accounting for storage costs

warehouse_capacity = {"A": 1000, "B": 4000}

demand_per_bar = {
    1: 500,
    2: 900,
    3: 1800,
    4: 200,
    5: 700,
}

total_demand_of_crates = sum(demand_per_bar.values())
total_supply_of_crates = sum(warehouse_capacity.values())

demand_per_bar[99] = total_supply_of_crates - total_demand_of_crates

demand_per_bar

{1: 500, 2: 900, 3: 1800, 4: 200, 5: 700, 99: 900}

In [103]:
# Problem definition 
model = lp.LpProblem("supply_cost_minimization",lp.LpMinimize)

# Variables - # of crates shipped from Warehouse X to Bar Y, with X in (A,B) and Y in (1,2,3,4,5)
x = lp.LpVariable.dicts("craters",[(warehouse, bar) for warehouse in warehouses for bar in bars], lowBound=0,cat='Integer')
x

{('A', 1): craters_('A',_1),
 ('A', 2): craters_('A',_2),
 ('A', 3): craters_('A',_3),
 ('A', 4): craters_('A',_4),
 ('A', 5): craters_('A',_5),
 ('A', 99): craters_('A',_99),
 ('B', 1): craters_('B',_1),
 ('B', 2): craters_('B',_2),
 ('B', 3): craters_('B',_3),
 ('B', 4): craters_('B',_4),
 ('B', 5): craters_('B',_5),
 ('B', 99): craters_('B',_99)}

In [104]:
# Suppose the storage cost of warehouse B (2) is more expensive than warehouse A (1)
transp_costs = dict(zip([(warehouse, bar) for warehouse in warehouses for bar in bars],
                        [2,4,5,2,1,1,3,1,3,2,3,2]))
transp_costs

{('A', 1): 2,
 ('A', 2): 4,
 ('A', 3): 5,
 ('A', 4): 2,
 ('A', 5): 1,
 ('A', 99): 1,
 ('B', 1): 3,
 ('B', 2): 1,
 ('B', 3): 3,
 ('B', 4): 2,
 ('B', 5): 3,
 ('B', 99): 2}

In [105]:
# Objective function to be minimized - total cost of transport
model += lp.lpSum([transp_costs[(wh,bar)] * x[(wh,bar)] for wh in warehouses for bar in bars])

In [106]:
# Supply Constraints
for wh in warehouses:
    model += lp.lpSum([x[(wh,bar)] for bar in bars]) <= warehouse_capacity[wh]

# Demand Constraints
for bar in bars:
    model += lp.lpSum([x[(wh,bar)] for wh in warehouses]) == demand_per_bar[bar]

In [107]:
status = model.solve()
print("Model solution status: ", lp.LpStatus[status])

Model solution status:  Optimal


In [108]:
optimal_solution = dict([(x[var].name, x[var].value()) for var in x])
optimal_solution

{"craters_('A',_1)": 0.0,
 "craters_('A',_2)": 0.0,
 "craters_('A',_3)": 0.0,
 "craters_('A',_4)": 0.0,
 "craters_('A',_5)": 700.0,
 "craters_('A',_99)": 300.0,
 "craters_('B',_1)": 500.0,
 "craters_('B',_2)": 900.0,
 "craters_('B',_3)": 1800.0,
 "craters_('B',_4)": 200.0,
 "craters_('B',_5)": 0.0,
 "craters_('B',_99)": 600.0}

Depending on how mnuch more expensive it is to store craters in each warehouse, the optimal solution changes

Le'ts see the effect of more demand than supply

In [113]:
warehouses = ["A", "B", "C"] # C is our dummy supplier that will meet the inbalance in demand and supply
bars = [1,2,3,4,5, 99] #99 is our dummy node accounting for storage costs

warehouse_capacity = {"A": 1000, "B": 3000} 

demand_per_bar = {
    1: 500,
    2: 900,
    3: 1800,
    4: 200,
    5: 700,
}

total_demand_of_crates = sum(demand_per_bar.values())
total_supply_of_crates = sum(warehouse_capacity.values())

# Balanced supply and demand
if (total_supply_of_crates - total_demand_of_crates == 0):
    demand_per_bar[99] = 0
    warehouse_capacity["C"] = 0
    
# More supply than demand
elif (total_supply_of_crates - total_demand_of_crates > 0):
    demand_per_bar[99] = total_supply_of_crates - total_demand_of_crates
    warehouse_capacity["C"] = 0

# More demand than supply
else:
    demand_per_bar[99] = 0
    warehouse_capacity["C"] = total_demand_of_crates - total_supply_of_crates
    

# Suppose the storage cost of warehouse B (2) is more expensive than warehouse A (1)
transp_costs = dict(zip([(warehouse, bar) for warehouse in warehouses for bar in bars],
                        [2,4,5,2,1,1,
                         3,1,3,2,3,2,
                         4,3,6,4,4,3]))

print(demand_per_bar)
print(warehouse_capacity)
print(transp_costs)

{1: 500, 2: 900, 3: 1800, 4: 200, 5: 700, 99: 0}
{'A': 1000, 'B': 3000, 'C': 100}
{('A', 1): 2, ('A', 2): 4, ('A', 3): 5, ('A', 4): 2, ('A', 5): 1, ('A', 99): 1, ('B', 1): 3, ('B', 2): 1, ('B', 3): 3, ('B', 4): 2, ('B', 5): 3, ('B', 99): 2, ('C', 1): 4, ('C', 2): 3, ('C', 3): 6, ('C', 4): 4, ('C', 5): 4, ('C', 99): 3}


In [114]:
# Problem definition 
model = lp.LpProblem("supply_cost_minimization",lp.LpMinimize)

# Variables - # of crates shipped from Warehouse X to Bar Y, with X in (A,B) and Y in (1,2,3,4,5)
x = lp.LpVariable.dicts("craters",[(warehouse, bar) for warehouse in warehouses for bar in bars], lowBound=0,cat='Integer')

# Objective function to be minimized - total cost of transport
model += lp.lpSum([transp_costs[(wh,bar)] * x[(wh,bar)] for wh in warehouses for bar in bars])

# Supply Constraints
for wh in warehouses:
    model += lp.lpSum([x[(wh,bar)] for bar in bars]) <= warehouse_capacity[wh]

# Demand Constraints
for bar in bars:
    model += lp.lpSum([x[(wh,bar)] for wh in warehouses]) == demand_per_bar[bar]
    
status = model.solve()
print("Model solution status: ", lp.LpStatus[status])

optimal_solution = dict([(x[var].name, x[var].value()) for var in x])
optimal_solution

Model solution status:  Optimal


{"craters_('A',_1)": 300.0,
 "craters_('A',_2)": 0.0,
 "craters_('A',_3)": 0.0,
 "craters_('A',_4)": 0.0,
 "craters_('A',_5)": 700.0,
 "craters_('A',_99)": 0.0,
 "craters_('B',_1)": 100.0,
 "craters_('B',_2)": 900.0,
 "craters_('B',_3)": 1800.0,
 "craters_('B',_4)": 200.0,
 "craters_('B',_5)": 0.0,
 "craters_('B',_99)": 0.0,
 "craters_('C',_1)": 100.0,
 "craters_('C',_2)": 0.0,
 "craters_('C',_3)": 0.0,
 "craters_('C',_4)": 0.0,
 "craters_('C',_5)": 0.0,
 "craters_('C',_99)": 0.0}

## Two stage production planning problem
- Decision maker must decide how to purchase material, labor and resources to produce end products and maximize provit
- GTC company produces wrenches and pliers
- Products are subject to availability of steel, machine capabilities (molding and assembly), labor and market demand

- GTC wants to determine how much steel to purchase
- Available assembly capacity and Product contribution to earnings are unknown presently, but will be known at the beginning of the next period
  - When it is known, it will determine how many wrenches and pliers to produce
  - Uncertainty is expressed as one of four possible scenarios, each one with equal probability

In [116]:
import pulp as lp
import numpy as np

In [119]:
# Values that doesn't change
products = ["wrenches","pliers"]
price = [130, 100]
steel = [1.5, 1]
molding = [1, 1]
assembly = [0.3, 0.5]
cap_steel = 27
cap_molding = 21
LB = [0, 0]
capacity_ub = [15, 16]
steel_price = 58

In [120]:
# Four scenarios
scenarios = [0, 1, 2, 3]
prob_scenario = [0.25, 0.25, 0.25, 0.25]
wrench_earnings = [160, 160, 90, 90]
plier_earnings = [100, 100, 100, 100]
cap_assembly = [8, 10, 8, 10]

In [121]:
# Combinations of products and scenarios

production = [(scen, prod) for scen in scenarios for prod in products]
price_scenario = [[wrench_earnings[scen], plier_earnings[scen]] for scen in scenarios]
price_items = [item for sublist in price_scenario for item in sublist]